# 01 – Descarga de datos de criptomonedas (BTC, ETH, LTC, DOGE)

Este notebook descarga las series diarias de precios de las criptomonedas:

- Bitcoin (BTC-USD)
- Ethereum (ETH-USD)
- Litecoin (LTC-USD)
- Dogecoin (DOGE-USD)

Periodo: 1 enero 2018 – 30 diciembre 2020.

Los datos se descargan desde Yahoo Finance utilizando la librería `yfinance` y se guardan en:

`../data/raw/criptomonedas_2018_2020_raw.csv`.

Este notebook forma parte del repositorio:

_“Análisis topológico de criptomonedas mediante paisajes de persistencia (SEIO 2023)”_.

In [ ]:
# ===============================================
# 1. Importación de librerías y configuración
# ===============================================

import os
from pathlib import Path

import pandas as pd
import yfinance as yf

print("Librerías importadas correctamente.")

## 2. Definición de parámetros y ruta de salida

Se definen los *tickers* de interés, el rango temporal y la ruta de salida para los datos crudos.

In [ ]:
# Diccionario de tickers: nombre corto -> ticker en Yahoo Finance
tickers = {
    "BTC": "BTC-USD",
    "ETH": "ETH-USD",
    "LTC": "LTC-USD",
    "DOGE": "DOGE-USD",
}

start_date = "2018-01-01"
end_date = "2020-12-30"

# Carpeta de salida para datos crudos
raw_dir = Path("../data/raw")
raw_dir.mkdir(parents=True, exist_ok=True)

output_file = raw_dir / "criptomonedas_2018_2020_raw.csv"
print("Los datos crudos se guardarán en:", output_file)

## 3. Descarga de series de precios desde Yahoo Finance

Para cada criptomoneda se descarga la serie histórica diaria. Dado que `yfinance` puede cambiar el formato de las columnas (por ejemplo, usar un `MultiIndex`), el código incluye una lógica robusta para seleccionar la columna de precio adecuada.

Orden de prioridad de columnas de precio:

1. `Adj Close`
2. `Close`
3. `Price`

In [ ]:
prices = {}

for name, ticker in tickers.items():
    print(f"\nDescargando {name} ({ticker})...")
    data = yf.download(
        ticker,
        start=start_date,
        end=end_date,
        auto_adjust=False
    )

    if data.empty:
        raise ValueError(f"No se obtuvieron datos para {ticker} en el rango especificado.")

    # Si las columnas son MultiIndex, nos quedamos con el primer nivel
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    # Seleccionamos la mejor columna de precio disponible
    selected_col = None
    for col in ["Adj Close", "Close", "Price"]:
        if col in data.columns:
            selected_col = col
            break

    if selected_col is None:
        raise ValueError(f"{ticker} no tiene columnas de precio apropiadas. Columnas encontradas: {list(data.columns)}")

    serie = data[selected_col].copy()
    prices[name] = serie

# Construimos un DataFrame con todas las criptomonedas
prices_df = pd.DataFrame(prices).dropna()
prices_df = prices_df.sort_index()

print("\nDimensiones del DataFrame de precios:", prices_df.shape)
prices_df.head()

## 4. Guardar datos crudos en `data/raw/`

Se exporta el DataFrame de precios a un archivo CSV para ser utilizado posteriormente
en el notebook de preprocesamiento (`02_preprocesamiento.ipynb`).

In [ ]:
prices_df.to_csv(output_file, index=True)
print(f"Datos crudos guardados correctamente en: {output_file}")

## 5. Verificación rápida (opcional)

Se muestra un resumen estadístico básico de las series de precios.

In [ ]:
summary = prices_df.describe().T
summary